In [ ]:
import openpyxl
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
import random
import time

workflow(0.042, 0.005, 20, 27.782, 1)
workflow(0.032, 0.004, 20, 8.437, 2)
workflow(0.032, 0.004, 20, 8.261, 3)
workflow(0.273, 0.018, 10, 5, 4)
workflow(0.273, 0.018, 10, 5, 5)
workflow(0.273, 0.018, 10, 5, 6)
workflow(0.133, 0.010, 1, 12.612, 7)
workflow(0.219, 0.020, 1, 8.888, 8)
workflow(0.159, 0.012, 1, 10.311, 9)
workflow(0.108, 0.009, 1, 6.751, 10)
workflow(0.133, 0.017, 1, 4.500, 11)




In [ ]:
def workflow(D, h, n, l, num):
    #reading data from datasheet
    
    wb = openpyxl.load_workbook(r"C:\Users\User\Desktop\Книга1.xlsx")
    sheet = wb['Лист3']

    temperature = []
    pressure = []
    flow = []
    t = 730
    i = 0
    for i in range (1, t):
        temperature.append(sheet.cell(row = i, column = 3).value+200)
        pressure.append(sheet.cell(row = i, column = 1).value)
        flow.append(sheet.cell(row = i, column = 5).value)
    #plt.plot(temperature)
    #plt.show()
    #plt.plot(pressure)
    #plt.show()
    #plt.plot(flow)
    #plt.show()

    #main set of constants

    d = D-2*h
    Pi = 3.1418
    Ro = 0.45
    Mup = 5.568
    ib = 829.1
    ie = 826.5
    f = ((Pi*(d**2))/4)*n
    ktp = 1.6
    Nut = 1.2
    Nup = 1
    Prsr = 0.94
    lambpr = 0.74
    mumr = 0.85
    time_meas = 50000

    #calculation of auxilary values

    RoG = flow[0]/f
    ReS = RoG*(D-2*h)/Mup
    XiP = (1.82*np.log10(ReS)-1.64)**-2
    dpp = XiP*(RoG**2)/(2*Ro)*l/(D-2*h)
    entp = ie + (Nut*Nup-1)*(ib-ie)
    ReSz = ((1.82*np.log10(ReS)-1.64)**-2)*(((ReS*Mup)**2)*l/(2*Ro*(D-2*h)))
    RomM = (ReS*Mup)/2*(D-2*h)
    qp = RomM*(ib-ie)*Nut*ktp*(d**2)/(4*D*l)
    Repr = RomM*d/Mup
    Xie = (1.82*np.log10(Repr)-1.64)**-2
    NuPR = (Xie*Repr*Prsr/8)/(1.07+12.7*pow((Xie/8), 1/2)*(pow(Prsr, 2/3)-1))
    alpr = NuPR*lambpr/d

    #calculating inner. middle and outer temperatures of heat exchanger pipes

    t_inner = temperature[0]+(mumr*D*qp)/(alpr*d)
    T_inner = t_inner+273.15
    #print(t_inner)

    t_medium = t_inner+(mumr*D*qp)/(4*lambpr)*np.log10(D/d)
    T_medium = t_medium+273.15
    #print(t_medium)

    t_outer = t_inner+(mumr*D*qp)/(2*lambpr)*np.log10(D/d)
    T_outer = t_outer+273.15
    #print(t_outer)

    #calculating thicness of corrosion layer 

    h_steam = 10**(1.49-(6000/T_inner)+0.4*np.log10(time_meas))
    #print(h_steam)

    h_gas = 10**(3.55-(7800/T_outer)+(0.4+0.143*(10**-4)*T_outer)*np.log10(time_meas))
    #print(h_gas)

    h_comm = h_steam+h_gas
    h_start = 1.08*h
    h_end = h_start-h_comm

    #calculating resource of heat exchanger pipes

    sigma_p = pressure[0]/2*((d/h_end)+1)

    tau_d = 10**(((24596-2400*np.log10(sigma_p)-10.9*sigma_p)/T_medium)+2*np.log10(T_medium)-24.88)
    #print(tau_d)
    res_p = tau_d/time_meas
    #print(res_p)
    res_ost = 720*tau_d*((0.8/(720*res_p)-1))
    #print(res_ost)

    #writing resource value into output file

    sheet.cell(row = num,column = 7).value = res_ost
    wb.save(r'C:\Users\User\Desktop\Книга1.xlsx')